In [1]:
from sklearn.ensemble import RandomForestClassifier
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.models import Sequential, Model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [28]:
data = []
data_dir = 'artifacts'
dirs = os.listdir(data_dir)
            
for i in dirs:
    path = os.path.join(data_dir,i)
    for img in os.listdir(path):
        image = cv.imread(os.path.join(path,img))
        image = cv.resize(image, (64,64))
        data.append([image, i])

In [29]:
images = []
labels = []

for img, lab in data:
    images.append(img)
    labels.append(lab)

In [30]:
from sklearn.preprocessing import LabelEncoder

In [31]:
images = np.array(images)

In [72]:
images = images/255.

In [52]:
le = LabelEncoder()
encoded = le.fit_transform(labels)

In [33]:
labels = np.array(labels)

In [12]:
from sklearn.model_selection import train_test_split

In [73]:
x_train, x_test, y_train, y_test = train_test_split(images, encoded, test_size=0.1)

In [132]:
inputs = Input(shape=(64,64,3))

x1 = (Conv2D(32, (2,2), input_shape=(64,64,3), padding="same", activation="relu"))(inputs)
x2 = (MaxPool2D(2,2))(x1)
x3 = (Conv2D(64, (2,2), padding="same", activation="relu"))(x2)
x4 = (MaxPool2D(2,2))(x3)
x5 = (Conv2D(128, (2,2), padding="same", activation="relu"))(x4)
x6 = (MaxPool2D(2,2))(x5)

x7 = (GlobalAveragePooling2D())(x6)
x8 = (Dense(100, activation="relu"))(x7)
x9 = (Dense(2, activation="sigmoid"))(x8)

In [135]:
model = Model(inputs=inputs, outputs=x9)

In [136]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        416       
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        8256      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       32896 

In [137]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [138]:
history = model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=40, batch_size=8)

Epoch 1/40
480/480 [==============================] - 19s 36ms/step - loss: 0.6757 - accuracy: 0.5969 - val_loss: 0.6722 - val_accuracy: 0.6019
Epoch 2/40
480/480 [==============================] - 22s 47ms/step - loss: 0.6719 - accuracy: 0.6008 - val_loss: 0.6653 - val_accuracy: 0.6066
Epoch 3/40
480/480 [==============================] - 21s 43ms/step - loss: 0.6591 - accuracy: 0.6034 - val_loss: 0.6558 - val_accuracy: 0.5948
Epoch 4/40
480/480 [==============================] - 24s 50ms/step - loss: 0.6486 - accuracy: 0.6222 - val_loss: 0.6511 - val_accuracy: 0.6042
Epoch 5/40
480/480 [==============================] - 22s 46ms/step - loss: 0.6412 - accuracy: 0.6435 - val_loss: 0.6511 - val_accuracy: 0.6089
Epoch 6/40
480/480 [==============================] - 25s 52ms/step - loss: 0.6358 - accuracy: 0.6386 - val_loss: 0.6458 - val_accuracy: 0.6230
Epoch 7/40
480/480 [==============================] - 23s 47ms/step - loss: 0.6309 - accuracy: 0.6472 - val_loss: 0.6480 - val_accuracy:

In [140]:
temp = Model(inputs=inputs, outputs=x6)

In [141]:
temp.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        416       
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        8256      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       3289

In [142]:
train_features = temp.predict(x_train)

120/120 [==============================] - 5s 37ms/step


In [143]:
train_features.shape

(3835, 8, 8, 128)

In [144]:
train_reshaped = train_features.reshape(train_features.shape[0], -1)

In [145]:
train_reshaped.shape

(3835, 8192)

In [147]:
test_features = temp.predict(x_test)
test_reshaped = test_features.reshape(test_features.shape[0], -1)

14/14 [==============================] - 1s 33ms/step


In [148]:
test_reshaped.shape

(427, 8192)

In [152]:
rf = RandomForestClassifier()
rf.fit(train_reshaped, y_train)

RandomForestClassifier()

In [153]:
y_pred_rf = rf.predict(test_reshaped)

In [154]:
accuracy_score(y_pred_rf, y_test)

0.9578454332552693

In [155]:
confusion_matrix(y_pred_rf, y_test)

array([[251,  12],
       [  6, 158]], dtype=int64)